In [1]:
import time
import numpy as np
import gym
import matplotlib.pyplot as plt

import sys
sys.path.append('../..')

import wizluk.envs
from wizluk.policies import ContinuousZeroPolicy, ContinuousRandomPolicy

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Parameters for Baselines

In [2]:
H = 100 # Horizon, 50 seconds of simulated time
N = 10 # Number of Rollouts
dt = 0.1

## Zero Policy

In [3]:
env = gym.make('ContinuousWalkBot-RandomInit-v1')
zero_baseline = np.zeros((N,1))

env.seed(1337)
zero_pi = ContinuousZeroPolicy()

for k in range(N):
    x = env.reset()
    for s in range(H):
        u = zero_pi.get_action(env, x)
        x, reward, done, info = env.step(u)
        zero_baseline[k] += reward
        if done : break

zero_baseline_cost = np.mean(zero_baseline)
print("Zero baseline cost: {}".format(zero_baseline_cost))

Zero baseline cost: -3929.835875230265


#### Demonstration

In [4]:
env = gym.make('ContinuousWalkBot-RandomInit-v1')
env.seed(1337)
zero_pi = ContinuousZeroPolicy()

x = env.reset()
for s in range(H):
    env.render()
    time.sleep(dt)
    u = zero_pi.get_action(env, x)
    x, reward, done, info = env.step(u)
    
    if done : break
env.close()

## Random Policy

In [5]:
env = gym.make('ContinuousWalkBot-RandomInit-v1')
np.random.seed(1337)
env.seed(1337)

random_baseline = np.zeros((N,1))
random_pi = ContinuousRandomPolicy()

for k in range(N):
    x = env.reset()
    for s in range(H):
        u = random_pi.get_action(env, x)
        x, reward, done, info = env.step(u)
        random_baseline[k] += reward
        if done : 
            break

random_baseline_cost = np.mean(random_baseline)
print("Random baseline cost: {}".format(random_baseline_cost))

Random baseline cost: -3433.5413504986127


#### Demonstration

In [6]:
env = gym.make('ContinuousWalkBot-RandomInit-v1')
np.random.seed(1337)
env.seed(1337)
random_pi = ContinuousRandomPolicy()

x = env.reset()
for s in range(H):
    env.render()
    time.sleep(dt)
    u = random_pi.get_action(env, x)
    x, reward, done, info = env.step(u)
    if done : break
env.close()

## Constrained LQR Finite Horizon Policy (via MIQP-based MPC controller)

In [7]:
from wizluk.envs.walker import WalkBot

env = gym.make('ContinuousWalkBot-RandomInit-v1')
np.random.seed(1337)
env.seed(1337)

[1337]

It's worth noting the knowledge we need to provide in order to approach this problem with a MPC controller:

 - A precise definition of the target (goal) state $x_G$.
 - The constraints on "inputs" (control variables, action) and "outputs" (the actual state variables).
 - The $Q$ and $R$ matrices.
 - The horizon for the Receding Horizon Control problem ($10$).
 - And full access to the system dynamics (discretised state and input matrices, matching the time step of the simulation)

In [8]:
x = env.reset()
x_G = np.matrix( [ [8.0], [2.0], [0.0], [0.0]])
model = WalkBot(1,'walker')

model.add_bounds('x', 0.0,10.0)
model.add_bounds('y', 0.0,10.0)
model.add_bounds('vx', -10.0,10.0)
model.add_bounds('vy', -10.0,10.0)
model.add_bounds('ax', -2.0,2.0)
model.add_bounds('ay', -2.0,2.0)

u = model.mpc( x, x_G, 10, np.eye(4), np.eye(2), dt=dt)
print(u)

[[1.9       ]
 [1.08453996]]


In [9]:
from wizluk.envs.walker import WalkBot

env = gym.make('ContinuousWalkBot-RandomInit-v1')
np.random.seed(1337)
env.seed(1337)

mpc_cost = 0.0


x = env.reset()
for s in range(H):
    env.render()
    time.sleep(dt)
    u = model.mpc( x, x_G, 10, np.eye(4), np.eye(2), dt=dt)
    #print(u)
    x, reward, done, info = env.step(u)
    mpc_cost += reward
    if done: break
env.close()

print('"Optimal" cost: {}'.format(mpc_cost))

"Optimal" cost: -493.84168627154423
